# Porting to ONNX

This notebook demonstrates how to port models from Transformers/PyTorch package to ONNX. It is based on the [Optimum](https://github.com/huggingface/optimum) library.

## Installation

We use [poetry](https://python-poetry.org/docs/cli) to manage dependencies. To install the dependencies, run:

```bash
poetry install
```

In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import os
from pathlib import Path
from typing import List, Tuple, Any

import numpy as np
import time
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
from optimum.onnxruntime import AutoOptimizationConfig, ORTModelForFeatureExtraction, ORTOptimizer, ORTModel
from optimum.pipelines import pipeline
import torch.nn.functional as F

In [3]:
# Load the tokenizer and PyTorch model from HuggingFace Transformers
model_id = "BAAI/bge-small-en-v1.5"
hf_model = AutoModel.from_pretrained(model_id)
hf_tokenizer = AutoTokenizer.from_pretrained(model_id)
quantize = True
if quantize:
    repository_id = f"Qdrant/{model_id.split('/')[1]}-onnx-Q"
else:
    repository_id = f"Qdrant/{model_id.split('/')[1]}-onnx"

In [4]:
# The input texts can be in any language, not just English.
# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
multilingual_queries = [
    "query: how much protein should a female eat",
    "query: 南瓜的家常做法",
    "query: भारत का राष्ट्रीय खेल कौन-सा है?",  # Hindi text
    "query: భారత్ దేశంలో రాష్ట్రపతి ఎవరు?",  # Telugu text
    "query: இந்தியாவின் தேசிய கோப்பை எது?",  # Tamil text
    "query: ಭಾರತದಲ್ಲಿ ರಾಷ್ಟ್ರಪತಿ ಯಾರು?",  # Kannada text
    "query: ഇന്ത്യയുടെ രാഷ്ട്രീയ ഗാനം എന്താണ്?",  # Malayalam text
]

english_texts = [
    "India: Where the Taj Mahal meets spicy curry.",
    "Machine Learning: Turning data into knowledge, one algorithm at a time.",
    "Python: The language that makes programming a piece of cake.",
    "fastembed: Accelerating embeddings for lightning-fast similarity search.",
    "Qdrant: The ultimate tool for high-dimensional indexing and search.",
]

In [5]:
def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


def hf_embed(model_id: str, inputs: List[str]):
    # Tokenize the input texts
    batch_dict = hf_tokenizer(inputs, max_length=512, padding=True, truncation=True, return_tensors="pt")

    outputs = hf_model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict["attention_mask"])

    # normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    return embeddings.detach().numpy()

In [6]:
hf_embed(inputs=english_texts, model_id=model_id)

array([[ 0.05485763,  0.08136623, -0.00395789, ...,  0.02512371,
        -0.03349504, -0.0593129 ],
       [ 0.01078518,  0.01582215,  0.04614557, ..., -0.01674951,
        -0.00244641, -0.06179965],
       [-0.06607923, -0.01235531, -0.00689854, ...,  0.10634594,
         0.12025263, -0.05135345],
       [-0.07568254,  0.00908228, -0.02221818, ...,  0.00177038,
        -0.0325426 ,  0.05233581],
       [-0.07008213,  0.02070545,  0.02720274, ..., -0.01158645,
        -0.01457597,  0.01262206]], dtype=float32)

## Load the model using ORTModelForFeatureExtraction

In [7]:
model = ORTModelForFeatureExtraction.from_pretrained(model_id, export=True)

# Load the optimization configuration detailing the optimization we wish to apply
if quantize:
    optimization_config = AutoOptimizationConfig.O3()
    optimizer = ORTOptimizer.from_pretrained(model)

save_dir = Path("local_cache") # Path to save the optimized model, working directory
optimizer.optimize(save_dir=save_dir, optimization_config=optimization_config)

Framework not specified. Using pt to export to ONNX.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Using framework PyTorch: 2.1.2
Overriding 1 configuration item(s)
	- use_cache -> False
/opt/homebrew/Caskroom/miniconda/base/envs/fst/lib/python3.11/site-packages/optimum/onnxruntime/configuration.py:770: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(
Optimizing model...
Configuration saved in local_cache/ort_config.json
Optimized model saved at: local_cache (external data format: False; saved all tensor to one file: True)


PosixPath('local_cache')

In [14]:
model = ORTModelForFeatureExtraction.from_pretrained(save_dir)
# optimized_model = AutoModel.from_pretrained(repository_id)
tokenizer = AutoTokenizer.from_pretrained(repository_id)
tokens = tokenizer(english_texts)
model(**tokens)

RuntimeError: Could not create tensor from given input list

In [ ]:
# model.push_to_hub(save_directory=save_dir, repository_id=repository_id, use_auth_token=True)

: 

: 

RuntimeError: Could not create tensor from given input list